<a href="https://colab.research.google.com/github/giyuntf2/1day1commit/blob/main/pytorch/MNIST_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple CNN

- first_layer = Convolutional Layer

(in_channel = 1, out_channel = 32, kernel_size = 3, stride = 1, padding = 1, activation_func = ReLU), 

(max_pooling = kernel_size = 2, stride =2)

- second_layer = Convolutional Layer

(in_channel = 32, out_channel = 64, kernel_size = 3, stride = 1, padding = 1, activation_func = ReLU), 

(max_pooling = kernel_size = 2, stride =2)

- third_layer = Fully-Connected Layer

(batch_szie x 7 x 7 x 64 => batch_size x 3136)


FCL = (nerual 10, activation function =  Softmax)

In [79]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

# conv layer def

In [82]:
conv1 = nn.Conv2d(1, 32, 3, padding=1)

In [83]:
conv1

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [84]:
conv2 = nn.Conv2d(32, 64, 3, padding=1)

In [85]:
conv2

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

# pooling layer def

In [87]:
pool = nn.MaxPool2d(2)

In [88]:
pool

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)

# Modeling

In [99]:
inputs = torch.Tensor(1, 1, 28, 28)

In [100]:
inputs.shape

torch.Size([1, 1, 28, 28])

In [101]:
out = conv1(inputs)

In [102]:
out.shape #32channel, 28 * 28 tensor

torch.Size([1, 32, 28, 28])

In [103]:
out = conv2(out)
out = pool(out)

In [104]:
out.shape

torch.Size([1, 64, 14, 14])

In [105]:
out = pool(out)

In [106]:
out.shape

torch.Size([1, 64, 7, 7])

# Tensor Flatten

In [114]:
for idx, dim in enumerate(range(0, 4)):
    print(f"{idx}번째 차원의 사이즈는 : {out.size(dim)}")

0번째 차원의 사이즈는 : 1
1번째 차원의 사이즈는 : 64
2번째 차원의 사이즈는 : 7
3번째 차원의 사이즈는 : 7


In [117]:
# .view()를 사용하면 Flatten 가능
out = out.view(out.size(0), -1)
print(out.shape)

torch.Size([1, 3136])


In [127]:
list(out.size())[1]

3136

# FCLayer

In [129]:
fc = nn.Linear(list(out.size())[1], 10)
out = fc(out)

In [131]:
out

tensor([[-0.0338, -0.0208,  0.0955, -0.0079,  0.0036, -0.0244, -0.0321,  0.0100,
         -0.0616,  0.0079]], grad_fn=<AddmmBackward0>)

In [130]:
out.shape

torch.Size([1, 10])

# CNN with Mnist

In [150]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn.init
from torch.utils.data import DataLoader

In [135]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(42)

if device == 'cuda':
    torch.cuda.manual_seed_all(42)

In [145]:
lr = 1e-3
epochs = 30
batch_size = 128

In [146]:
mnist_train = datasets.MNIST(root='data',
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)

mnist_test = datasets.MNIST(root='data',
                            train=False,
                            transform=transforms.ToTensor(),
                            download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [167]:
data_loader = DataLoader(dataset=mnist_train,
           batch_size=batch_size,
           shuffle=True,
           generator=torch.Generator(device='cuda')
)

In [196]:
class MNIST_CNN(nn.Module):
    def __init__(self):
        super(MNIST_CNN, self).__init__()

        self.c1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.c2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias = True)

        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, input):
        out = self.c1(input)
        out = self.c2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)

        return out
        

In [197]:
model = MNIST_CNN().to(device)

In [198]:
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 469


In [199]:
loss_function = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [201]:
from torchsummary import summary
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
              ReLU-2           [-1, 32, 28, 28]               0
         MaxPool2d-3           [-1, 32, 14, 14]               0
            Conv2d-4           [-1, 64, 14, 14]          18,496
              ReLU-5           [-1, 64, 14, 14]               0
         MaxPool2d-6             [-1, 64, 7, 7]               0
            Linear-7                   [-1, 10]          31,370
Total params: 50,186
Trainable params: 50,186
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.65
Params size (MB): 0.19
Estimated Total Size (MB): 0.84
----------------------------------------------------------------


In [200]:
for ep in range(epochs):
    avg_cost = 0
    for X, y in data_loader:
        X = X.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = loss_function(hypothesis, y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(ep + 1, avg_cost))

[Epoch:    1] cost = 0.249538422
[Epoch:    2] cost = 0.0620332956
[Epoch:    3] cost = 0.0453522503
[Epoch:    4] cost = 0.0364452675
[Epoch:    5] cost = 0.0303604528
[Epoch:    6] cost = 0.027050782
[Epoch:    7] cost = 0.0213003028
[Epoch:    8] cost = 0.01787409
[Epoch:    9] cost = 0.016231183
[Epoch:   10] cost = 0.0130706066
[Epoch:   11] cost = 0.0113979774
[Epoch:   12] cost = 0.00991098862
[Epoch:   13] cost = 0.00822579022
[Epoch:   14] cost = 0.00709168473
[Epoch:   15] cost = 0.00678944169
[Epoch:   16] cost = 0.0057221693
[Epoch:   17] cost = 0.00439953478
[Epoch:   18] cost = 0.00440857559
[Epoch:   19] cost = 0.00405650493
[Epoch:   20] cost = 0.00488419784
[Epoch:   21] cost = 0.00266772532
[Epoch:   22] cost = 0.0028433213
[Epoch:   23] cost = 0.00226497441
[Epoch:   24] cost = 0.000921934959
[Epoch:   25] cost = 0.000828718184
[Epoch:   26] cost = 0.00461994205
[Epoch:   27] cost = 0.00514532439
[Epoch:   28] cost = 0.000988085289
[Epoch:   29] cost = 0.000231319456

In [180]:
# 학습을 진행하지 않을 것이므로 torch.no_grad()
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.988599956035614


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
